# Mini project: Simulating Pac-Man

In the last practical session, we saw how run multiple behaviors in parallel on multiple robots. In this section we will see more advanced methods for combining behaviors by modulating behavior activations according to internal states of the robot and by allowing them to sense the attributes of others. In order to start with a clean basis, let's first provide the definition of several behaviors. These four behaviors are simple implementations of the [Braitenberg vehicles](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame1550) we have seen in class, where the sources (i.e. what is sensed by the proximeters) are other epucks:

In [ ]:
from numpy.linalg import norm
from numpy import array

from simulator_interface import open_session, close_session
simulator, epuck1 = open_session(n_epucks=1)

# First reduce the maximum speed
epuck1.max_speed = 3

def line_following(robot):
    floor_left, floor_middle, floor_right = robot.floor_sensor()
    # if can go right
    if floor_middle == 1.0 and floor_right == 1.0 and robot.rotate == 'right':
        return  [1.0, 0]
    # if can go left
    if floor_middle == 1.0 and floor_left == 1.0 and robot.rotate == 'left':
        return  [0, 1.0]
    if floor_middle == 1.0: return [.3, .3]
    if floor_left == 0.0 and floor_right == 0.0: return [0.1, 0]
    return [floor_right/2.0, floor_left/2.0]

def detect_food(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    if (left > 0 or right > 0):
        robot.rotate = 'left' if left > right else 'right'
        
# Reverse-engineered part of the code in order to make spheres disappear when eaten   
def eat(robot):            
    robot_pos = array(robot.position())
    for obj in simulator.eatable_objects:
        try:
            obj_pos = array(simulator.get_object_position(obj))
        except:
            break
        if norm(robot_pos - obj_pos) < 0.1:
            simulator.eatable_objects.remove(obj)
            simulator.remove_object(obj)

simulator.eatable_objects = ['Sphere_1', 'Sphere_2', 'Sphere_3', 'Sphere_4', 'Sphere_5']
epuck1.attach_routine(eat, freq=1)
epuck1.start_routine(eat)
########

epuck1.detach_all_behaviors()
epuck1.rotate = 'no'
epuck1.attach_behavior(line_following, freq=10)
epuck1.attach_routine(detect_food, freq=1)
epuck1.start_routine(detect_food)
epuck1.start_all_behaviors()
    

In [ ]:
close_session(simulator)